# Thermal Models

ACISpy provides the ability to run [Xija](http://cxc.cfa.harvard.edu/mta/ASPECT/tool_doc/xija/index.html) thermal models via a special class, `ThermalModelRunner`, which can input commanded states from a variety of sources. The really nice thing about `ThermalModelRunner` is that it is actually a `Dataset` object, so we can look at the different fields, make plots, and create derived fields.

In [ ]:
import acispy
import numpy as np

## Basic Use of `ThermalModelRunner`

If a thermal model is being run entirely within the past or another situation where `kadi` commanded states are fully specified, you can simply run a thermal model by specifying the MSID to be modeled and the start and stop times of the model. The initial temperature will be specified from telemetry unless one sets an initial value using the `T_init` keyword argument. If we want to take model "bad times" into account, we also need to set `mask_bad_times=True`:

In [ ]:
dea_model = acispy.ThermalModelRunner("1deamzt", "2020:142", "2020:150", 
                                      get_msids=True, mask_bad_times=True)

`get_msids=True` is set, so the resulting `Dataset` object will be populated with actual telemetry as well as model data, so that you can examine and plot both:

In [ ]:
print(dea_model["model","1deamzt"][:10])
print(dea_model["msids","1deamzt"][:10])

In [ ]:
dp = acispy.DatePlot(dea_model, [("msids", "1deamzt"), ("model", "1deamzt")], plot_bad=True)
dp

Note that for this period of time a segment of it was marked as a "bad time" in the model, and this shows up in the cyan region in the plot. 

By default, model specification files will be searched for in the `chandra_models` package. If your model spec file is not in that package, or you want to specify a different model specification file, you can pass `ThermalModelRunner` a `model_spec` optional argument:

In [ ]:
dea_model2 = acispy.ThermalModelRunner("1deamzt", "2020:142", "2020:150",
                                       model_spec="/Users/jzuhone/Source/dea_check/dea_check/dea_model_spec.json")

## Running Thermal Models Using States from Various Sources

The `ThermalModelRunner` class also has various methods which one can use to run thermal models using states from various sources.

### Running a Thermal Model from Commands

You can use a set of commands (either a `kadi` `CommandTable` or list of command dicts) to run a thermal model using the `from_commands` method:

In [ ]:
from kadi import commands
# commands as a CommandTable
cmds = commands.get_cmds('2018:001:00:00:00', '2018:002:00:00:00')
psmc_model = acispy.ThermalModelRunner.from_commands("1pdeaat", cmds)
# commands as a list of dicts
dict_cmds = cmds.as_list_of_dict()
psmc_model2 = acispy.ThermalModelRunner.from_commands("1pdeaat", dict_cmds)

### Running a Thermal Model from a Backstop File

You can use a backstop file as input to `ThermalModelRunner` to generate states for a thermal model run. In this case, `ThermalModelRunner` will go back to the latest telemetry to determine the initial temperature (unless one is specified, see below) and construct states from that point and handle continuity between those and the commands in the backstop file appropriately. To show this, we'll use the ACA thermal model, which requires that the initial value for the `aca0` pseudonode be specified, which we can do using the `other_init` keyword argument:

In [ ]:
tm_aca = acispy.ThermalModelRunner.from_backstop(
    "aacccdpt", "/data/acis/LoadReviews/2020/AUG1720/ofls/CR229_2202.backstop",
    other_init={"aca0": -10})

In [ ]:
dp = acispy.DatePlot(tm_aca, ('model','aacccdpt'), field2="pitch")
dp

### Running a Thermal Model Using a `states.dat` File

One can run a thermal model from a `states.dat` table file which would be outputted by the thermal model check scripts during a load review, using the `from_states_file()` method:

In [ ]:
dpa_model = acispy.ThermalModelRunner.from_states_file("1dpamzt", "my_states.dat")

### Running Thermal Models Using States Constructed By Hand

You can create a dictionary of states completely by hand and submit them as a keyword argument to `ThermalModelRunner`, for running completely hypothetical thermal models. For simplicity, we'll pick constant states except change the CCD count.

In [ ]:
states = {"ccd_count": np.array([5,6,1]),
          "pitch": np.array([150.0]*3),
          "fep_count": np.array([5,6,1]),
          "clocking": np.array([1]*3),
          "vid_board": np.array([1]*3),
          "off_nom_roll": np.array([0.0]*3),
          "simpos": np.array([-99616.0]*3),
          "datestart": np.array(["2015:002:00:00:00","2015:002:12:00:00","2015:003:12:00:00"]),
          "datestop": np.array(["2015:002:12:00:00","2015:003:12:00:00","2015:005:00:00:00"])}

In the previous examples, we never specified an initial temperature, but we can always do so, and must do so if we are past the end value in telemetry. This is not the case here, but we'll set one anyway:

In [ ]:
T_init = 13.0 # in degrees C

Now we can pass in the `states` dict we created, as well as the `T_init`:

In [ ]:
dea_model = acispy.ThermalModelRunner("1deamzt", "2015:002:00:00:00", 
                                      "2015:005:00:00:00", states=states, T_init=T_init)

In [ ]:
dp = acispy.DatePlot(dea_model, ("model","1deamzt"), field2="ccd_count")
dp.set_ylim2(0,7)
dp

We can also dump the results of the model run to disk, both the states and the model components:

In [ ]:
dea_model.write_model("model.dat", overwrite=True)
dea_model.write_states("states.dat", overwrite=True)

These files can be loaded in at a later date using `ModelDataFromFiles`.

## Making Dashboard Plots

### NOTE: This functionality requires the `xijafit` package to be installed. 

It is possible to use the thermal model objects and the `xijafit` package to make dashboard plots. For this, use the `make_dashboard_plots()` method:

In [ ]:
dpa_model_long = acispy.ThermalModelRunner("1dpamzt", "2016:200", "2017:200", 
                                           mask_bad_times=True, get_msids=True)
dpa_model_long.make_dashboard_plots("1dpamzt", figfile="my_dpa_dash.png")

`figfile` sets the filename to save the dashboard plot to.

One can also use the `errorplotlimits` and `yplotlimits` arguments to set the bounds of the temperature and the errors on the plots:

In [ ]:
fp_model_long = acispy.ThermalModelRunner("fptemp_11", "2016:200", "2017:200", 
                                          mask_bad_times=True, get_msids=True)
fp_model_long.make_dashboard_plots("fptemp_11", yplotlimits=(-120.0, -104.0), 
                                   errorplotlimits=(-5.0, 5.0))

## Plotting Pitch and State Power Heating Values

All thermal models have a solar heating component. To make a quick solar heating plot, use the `make_solarheat_plot` method, with the node that the solar heating component acts upon as the first argument. 

In [ ]:
dpa_model_long.make_solarheat_plot("dpa0", figfile="dpa0_pitches.png")

Similarly, to plot the ACIS state power coefficients for a model (if present), use the `make_power_plot` method:

In [ ]:
# For the 1DEAMZT model set use_ccd_count=True
dpa_model_long.make_power_plot(figfile="acis_state_power.png", use_ccd_count=False)

## Simulating Single States

The `SimulateSingleState` class is a simplified implemenation of `ThermalModelRunner` which assumes that the spacecraft state is constant over a period of time. This class takes the name of the temperature to simulate, a start and stop date, and a dictionary of states. It is not required to specify all states in this dictionary, but note that states not specified will be assumed to be the following:

* `pitch`: 90 deg
* `off_nom_roll`: 0 deg
* `ccd_count`: 0
* `fep_count`: 0
* `clocking`: 0
* `vid_board`: 0
* `simpos`: -99616.0
* `hetg`: `"RETR"`
* `letg`: `"RETR"`
* `dh_heater`: 0
* `q1`: 1.0
* `q2`: 0.0
* `q3`: 0.0
* `q4`: 0.0 

In [ ]:
datestart = "2019:181:01:00:00" # start time of run
datestop = "2019:183:01:00:00" # stop time of run
states = {"pitch": 70.0}
T_init = -11.0 # in degrees F 
aca_m = acispy.SimulateSingleState("aacccdpt", datestart, datestop, states, T_init, other_init={"aca0": T_init})

In [ ]:
dp = acispy.DatePlot(aca_m, "aacccdpt")
dp

### Simulating ECS Runs

A further special case of running a single state is an ECS run, which may be performed after a safing action. The `SimulateECSRun` class is a limited version of `SimulateSingleState` which assumes that the SIM-Z position is HRC-S (-99616) and that ACIS is clocking. The goal is to predict if the temperature will hit the planning limit within the time frame of the ECS run. A start time is specified, as well as the number of `hours` in the ECS run (the same number of hours as specified on the ECS CAP). Then, an `attitude` must be specified, which can take one of three forms:
* A (pitch, off-nominal roll) combination
* An attitude quaternion
* The name of a load (for vehicle load ECS runs)

We pick a model to run (most relevant is `"1dpamzt"`) along with a start time a length of the ECS run in hours (24 in this case), and feed the parameters into `SimulateECSRun`:

In [ ]:
datestart = "2015:002:00:00:00" # start time of run
hours = 24 # length of ECS run in hours
attitude = [137.0, 0.0] # pitch, off-nominal roll in degrees
T_init = 7.5 # in degrees C
ccd_count = 6 # number of CCDs
dpa_ecs_run = acispy.SimulateECSRun("1dpamzt", datestart, hours, T_init, attitude, 
                                    ccd_count)

*NOTE* that the actual length of the ECS run, as per the ECS CAP, is `hours`+ 10 ks + 12 s. 

The run reports back the input parameters and the time when the limit was reached, if it was at all. We can plot the model using the `plot_model()` method, 
which shows the limit value as a dashed green line and the time at which the limit was reached as a dashed red line, as well as whether or not this was a safe ECS run:

In [ ]:
dpa_ecs_run.plot_model()

On the other hand, if the ECS run had been shorter, the limit would be reached _after_ the ECS run, so this is safe.

In [ ]:
hours = 14
dpa_ecs_run = acispy.SimulateECSRun("1dpamzt", datestart, hours, T_init, attitude, 
                                    ccd_count)
dpa_ecs_run.plot_model()

Note that for some combinations of parameters the limit may never be reached. For example, let's knock the CCD count down to 4:

In [ ]:
ccd_count = 4 # only 4 CCDs
dpa_ecs_run = acispy.SimulateECSRun("1dpamzt", datestart, hours, T_init, attitude,
                                    ccd_count)
dpa_ecs_run.plot_model()

### Simulating ECS Runs with Vehicle Loads

If the spacecraft executed SCS-107, we may be running an ECS run while the vehicle load is still running, which means that the pitch and off-nominal roll may change during the ECS run. If this is the case, pass the name of the load to the ``attitude`` parameter. An example of an ECS run with a vehicle load that is not safe:

In [ ]:
datestart = "2017:256:03:20:00"
hours = 24
T_init = 12.0 # in degrees C
attitude = "SEP0917C" # attitude info will be take from this load
ccd_count = 6 # number of CCDs 
dpa_ecs_run = acispy.SimulateECSRun("1dpamzt", datestart, hours, T_init, attitude, ccd_count)
dpa_ecs_run.plot_model()

But if we drop it down to 5 chips, it is safe:

In [ ]:
ccd_count = 5 # number of CCDs 
dpa_ecs_run = acispy.SimulateECSRun("1dpamzt", datestart, hours, T_init, attitude, ccd_count)
dpa_ecs_run.plot_model()

### Simulating the Focal Plane Temperature for ECS Runs 

The ACIS Focal Plane (FP) temperature can also be modeled for an ECS run. This will track the amount of "cold time" for the ECS measurement, as well as determine if the run will exceed the FP limit for that particular instrument, ACIS-I or ACIS-S. In this case, a quaternion generally should be supplied for the value of ``attitude``, since the FP temperature can depend on the value of the Earth solid angle. The ``instrument`` argument should also be supplied to determine what thermal limit will be applied.  

In [ ]:
datestart = "2020:148:14:45:00" # start time of run
hours = 24 # length of ECS run in hours
T_init = -115.0 # in degrees C
ccd_count = 4 # number of CCDs
attitude = [-0.04470333, 0.63502552, -0.67575906, 0.37160988] # attitude quaternion required
instrument = "ACIS-S"

fp_ecs_run = acispy.SimulateECSRun("fptemp_11", datestart, hours, T_init, attitude, ccd_count,
                                   instrument=instrument)
fp_ecs_run.plot_model()

This can also be done for a vehicle load as the ``attitude`` parameter:

In [ ]:
datestart = "2017:256:03:20:00"
hours = 24
T_init = -119.0 # in degrees C
attitude = "SEP0917C" # attitude info will be take from this load
ccd_count = 6 # number of CCDs 
fp_ecs_run = acispy.SimulateECSRun("fptemp_11", datestart, hours, T_init, attitude, ccd_count, 
                                    instrument="ACIS-I")
fp_ecs_run.plot_model()